In [1]:
files_directory = "C:/Users/g.nenna/OneDrive - Reply/Documents/GitHub/hakapizza/Hackapizza Dataset/Menu/"
#open each pdf file in the directory
import os
from PyPDF2 import PdfReader
import re
import json
import pandas as pd

def read_pdf_to_string(file_path):
    reader = PdfReader(file_path)
    file_title = file_path.split("/")[-1]
    content = "file_title: " + file_title + "\n"
    for page in reader.pages:
        content += page.extract_text() + "\n"  # Combine text from all pages
    return content

def get_txt_files(directory):
    txt_files = []
    for file in os.listdir(directory):
        #print(file)
        if file.endswith(".pdf"):
            txt_files.append(read_pdf_to_string(directory + file))
    return txt_files


In [2]:
documents = get_txt_files(files_directory)

incorrect startxref pointer(1)
incorrect startxref pointer(1)
incorrect startxref pointer(1)


In [3]:
from utils import *
from langchain_ibm import ChatWatsonx
from ibm_watsonx_ai.foundation_models import ModelInference
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from typing import List, Dict, Annotated
from langchain_openai import ChatOpenAI
import os


os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
llm_openai = ChatOpenAI(
    model="gpt-4o",
    temperature=0.01,
    seed = 420
)
llm_openai.invoke([HumanMessage("hello")]).content

'Hello! How can I assist you today?'

In [4]:
def get_sql_schemas(documents, suggestions, llm) -> str:
    schemas = ""
    first_prompt = """You are given a set of documents that should have some things in common (e.g. they are all menus, or they are all resumes). You need to extract the schema of the data in these documents. The schema is a list of tables, each with a list of columns. In the end You need to extract a set of schemas of sql tables that can be populated with informations from each document. The set of sql tables should be enough to execute queries that can extract informations from the documents. Don't use primary or foreign keys, the tables should be independent and should be max 4. DONT USE PRIMARY OR FOREIGN KEYS. duplicate information instead of using foreign keys. Use these table names:
    {table_suggestions}
    DOCUMENTS:
    {documents}"""

    documents_str = ""
    for index, document in enumerate(documents[0:20]):
        documents_str += f"##############Document {index + 1}: {document}\n\n"
    
    first_prompt = first_prompt.format(documents=documents_str, table_suggestions=suggestions)
    llm_output = llm.invoke([HumanMessage(first_prompt)]).content
    return llm_output
    

In [5]:
suggestions = ["restaurants", "dishes"]
schemas_string = get_sql_schemas(documents, suggestions, llm_openai)

In [6]:

def get_sql_instruction(schemas_string, llm_openai):
    """given a description of the schemas create a set of sql instructions to create the tables"""
    sql_instructions = ""
    first_prompt = """You are given a set of schemas of sql tables. You need to formulate the SQL query that can create these tables all at once. The query should be a string that can be executed in a SQL database and should create all the tables in the schema. The query must be semantically correct and should not contain any syntax errors. Return ONLY THE SQL CODE. Do not include any other information or comments.
    
    SCHEMAS:
    {schemas}"""

    first_prompt = first_prompt.format(schemas=schemas_string)
    llm_output = llm_openai.invoke([HumanMessage(first_prompt)]).content
    return llm_output.replace("`", "").replace("sql", "")
    

In [7]:
sql_instructions = get_sql_instruction(schemas_string, llm_openai)
print(sql_instructions)


CREATE TABLE restaurants (
    restaurant_name VARCHAR(255),
    chef_name VARCHAR(255),
    location VARCHAR(255),
    description TEXT,
    skills TEXT,
    menu TEXT
);

CREATE TABLE dishes (
    dish_name VARCHAR(255),
    restaurant_name VARCHAR(255),
    ingredients TEXT,
    techniques TEXT,
    description TEXT
);



In [8]:
import sqlite3
def create_tables(sql_instructions):
    statements = sql_instructions.split(";")
    conn = sqlite3.connect('knowledge_base.db')
    c = conn.cursor()
    for statement in statements:
        c.execute(statement)
    conn.commit()
    conn.close()


In [9]:
create_tables(sql_instructions)

In [10]:
import sqlite3
def get_actual_schema():
# Connessione al database
    conn = sqlite3.connect('knowledge_base.db')
    cur = conn.cursor()

# Ottenere l'elenco delle tabelle
    cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tabelle = cur.fetchall()
    return_str = ""
# Per ogni tabella, estrai lo schema
    for tabella in tabelle:
        nome_tabella = tabella[0]
        return_str += f"Schema della tabella: {nome_tabella}\n"
        
        # Ottenere lo schema della tabella
        cur.execute(f"PRAGMA table_info({nome_tabella});")
        schema = cur.fetchall()
        
        # Stampare lo schema
        for colonna in schema:
            return_str += str(colonna) + "\n"

    # Chiudi la connessione
    conn.close()
    return return_str

In [11]:
def get_rows_from_document(document, llm_openai, actual_schema):
    prompt = """"
    You are given a document and schemas of multiple SQL tables. Given the document you have to return the rows that can be inserted into the tables. The tables are independent and you can insert the rows in any order. You can return multiple rows to be inserted in multiple tables. Structure the output as follows:
    Rows to be inserted in Table 1:
    rows for table 1
    Rows to be inserted in Table 2:
    rows for table 2
    ...

    The document is:
    {document}
    the json schema to be used for reference is:
    {actual_schema}
    """

    prompt = prompt.format(document=document, actual_schema=actual_schema)
    llm_output = llm_openai.invoke([HumanMessage(prompt)]).content
    return llm_output

In [12]:
rows_string = get_rows_from_document(documents[0], llm_openai, get_actual_schema())

In [13]:
def get_sql_instruction_to_populate(rows_string, actual_sql_schema, llm_openai):
    prompt= """
    You are given a string that contains rows that can be inserted into multiple SQL tables. You are also given the schema of the SQL tables. You need to formulate the SQL query that can insert these rows into the tables. The query should be a string that can be executed in a SQL database and should insert all the rows in the schema. The query must be semantically correct and should not contain any syntax errors. Return ONLY THE SQL CODE. Do not include any other information or comments. Remember that strings should be enclosed in single quotes in SQL queries. When you have to use a single quote inside a string, you should escape it with another single quote. For example, if you have to insert the string "It's a beautiful day", you should write 'It''s a beautiful day'.

    ROWS:
    {rows}
    SCHEMA:
    {schema}
"""
    prompt = prompt.format(rows=rows_string, schema=actual_sql_schema)
    llm_output = llm_openai.invoke([HumanMessage(prompt)]).content
    return llm_output.replace("`", "").replace("sql", "").replace("\\", "")


In [14]:
sql_instruction_to_populate = get_sql_instruction_to_populate(rows_string, get_actual_schema(), llm_openai)
print(sql_instruction_to_populate)


INSERT INTO restaurants (restaurant_name, chef_name, location, description, skills, menu) VALUES ('Anima Cosmica', 'Aurora Stellaris', 'Pandora', 'Nel cuore pulsante di Pandora, dove le foreste bioluminescenti e le montagne fluttuanti si uniscono in un mosaico vibrante, emerge un gioiello culinario che fonde la magia di questo mondo alieno con l''eccellenza gastronomica. L''Anima Cosmica, guidato dall''innovativa Chef Aurora Stellaris, è il vertice della cucina pandoriana, una sintesi di tradizione Na''vi e avanzata scienza culinaria.', 'Psionica III, Temporale I, Gravitazionale I, Antimateria I, Quantistica 16, Luce II, Gradi di influenza di livello tecnologico II', 'Nebulosa Celestiale alla Stellaris, Sinfonia Cosmica ma Fatta Male, Astro-Risotto alle Onde Temporali, Sinfonia Galattica alla Griglia Cangiante, Galassie Infiammate: Sinfonia Cosmica in Sei Dimensioni, Galassie Sospese: Un Viaggio di Sapori Cosmogalattici, Pioggia di Andromeda, Sinfonia Cosmica ma Fatta Bene, Odissea Co

In [15]:
def populate_tables(sql_instruction_to_populate):
    statements = sql_instruction_to_populate.split(";")
    conn = sqlite3.connect('knowledge_base.db')
    c = conn.cursor()
    for statement in statements:
        c.execute(statement)
    conn.commit()
    conn.close()
    
populate_tables(sql_instruction_to_populate)

In [16]:
def populate_all_tables(documents, llm_openai):
    for document in documents:
        try:
            rows_string = get_rows_from_document(document, llm_openai, get_actual_schema())
            sql_instruction_to_populate = get_sql_instruction_to_populate(rows_string, get_actual_schema(), llm_openai)
            populate_tables(sql_instruction_to_populate)
        except Exception as e:
            print(e)
            continue
populate_all_tables(documents, llm_openai)

unrecognized token: "'ATTO I: Galassia di Sapori Eterei, Il Risveglio del Drago Celeste, L''Abbraccio del Cosmo"
near "universo": syntax error


In [17]:
def get_expert_agent_prompt(documents, llm):
    prompt = """
    You will read a lot of documents. You need to extract common information between those documents and produce a concise description of the document collection. The description will be used to instruct an agent that will answer questions about the documents translating queries in sql using the schema:
    {sql_schema}.

    DOCUMENTS:
    {documents}
    """
    documents_str = ""
    for index, document in enumerate(documents[0:20]):
        documents_str += f"##############Document {index + 1}: {document}\n\n"
    prompt = prompt.format(documents=documents_str, sql_schema=get_actual_schema())
    llm_output = llm.invoke([HumanMessage(prompt)]).content
    return llm_output


In [81]:
def translate_natural_language_into_sql(natural_language_query, actual_sql_schema, llm_openai):
    prompt = """
    You are given a natural language query that you need to translate into SQL. The query should be a string that can be executed in a SQL database and should return the correct results. The query must be semantically correct and should not contain any syntax errors. Return ONLY THE SQL CODE. Do not include any other information or comments. Avoid Joins. The query has to be executed on the following schema:
    {schema}

    QUERY:
    {query}
    """
    prompt = prompt.format(query=natural_language_query, schema=actual_sql_schema)
    llm_output = llm_openai.invoke([HumanMessage(prompt)]).content
    return llm_output.replace("`", "").replace("sql", "").replace("\\", "")

In [82]:
def get_sql_results(sql_query):
    conn = sqlite3.connect('knowledge_base.db')
    c = conn.cursor()
    for statement in sql_query.split(";"):
        c.execute(statement)
    results = c.fetchall()
    conn.close()
    return results


In [25]:
tmp = get_expert_agent_prompt(documents, llm_openai)

In [76]:
conn = sqlite3.connect('knowledge_base.db')
c = conn.cursor()
c.execute("select count(*) FROM dishes;")
results_test = c.fetchall()
conn.close()
print(results_test)

[(278,)]


In [83]:
## expert agent creation
@tool
def query_the_kb(natural_language_query: str) -> str:
    """tool used to query the knowledge base. The tool takes a natural language query and returns the results of the query. It uses SQL internally so the results will be shown in a tabular form"""
    sql_query = translate_natural_language_into_sql(natural_language_query, get_actual_schema(), llm_openai)
    print(sql_query)
    results = get_sql_results(sql_query)
    return results

expert_agent_prompt =" You are an expert in retrieving information from a document collection. These are the details about the document collection: " + tmp

expert_agent_1 = create_react_agent(llm_openai, [query_the_kb], state_modifier=SystemMessage(expert_agent_prompt))

In [34]:
expert_agent_1.invoke({'messages': [('user', ' Quali sono i piatti che includono le Chocobo Wings come ingrediente?')]})


SELECT dish_name 
FROM dishes 
WHERE ingredients LIKE '%Chocobo Wings%'



{'messages': [HumanMessage(content=' Quali sono i piatti che includono le Chocobo Wings come ingrediente?', additional_kwargs={}, response_metadata={}, id='d5b4cecc-fdde-4f71-a030-14151f465ecd'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_dhI19EkQpSOVUtii2RpoFzna', 'function': {'arguments': '{"natural_language_query":"Piatti che includono le Chocobo Wings come ingrediente"}', 'name': 'query_the_kb'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 572, 'total_tokens': 601, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_50cad350e4', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-991e3ba1-9451-4dc4-af4e-c5081cebaced-0', tool_calls=[{'name': 'query_the_kb', 'args': {

## secondo agente

In [47]:
files_directory = "C:/Users/g.nenna/OneDrive - Reply/Documents/GitHub/hakapizza/Hackapizza Dataset/Codice Galattico/"
document_agent_2 = get_txt_files(files_directory)[0]


In [48]:
@tool
def query_document(query):
    """query the document and return the relevant information"""
    prompt = """ given a query about a document, return the relevant information
    DOCUMENT:
        {document}
    QUERY:
        {query}
    """
    #print(prompt.format(document=document_agent_2, query=query))
    return llm_openai.invoke([HumanMessage(prompt.format(document=document_agent_2, query=query))]).content

document_agent_2_prompt = """You are an expert in extracting information from a document that has the following characteristics: . **Definizione degli Ordini**: per ogni Ordine di appartenenza c'è una regola precisa
2. **Sostanze Regolamentate**:
   - Identificate cinque macro-categorie di sostanze regolamentate, tra cui sostanze psicotrope, mitiche, xenobiologiche, quantiche e dimensionali, e spazio-temporali.
   - Ogni categoria ha regolamenti specifici sui limiti di concentrazione basati su caratteristiche particolari come il Coefficiente di Risonanza Psionica (CRP), l'Indice di Purezza Mitica (IPM), e altre misure.
3. **Licenze di Preparazione:**
   - Sono istituiti diversi tipi di licenze necessarie per eseguire vari metodi di preparazione del cibo
   - Le licenze variano per tipologia e livello, e prevedono requisiti tecnologici specifici.
   """

expert_agent_2 = create_react_agent(llm_openai, [query_document], state_modifier=SystemMessage(document_agent_2_prompt))


In [ ]:
expert_agent_2.invoke({'messages': [('user', 'info su licenza assionica')]})

### terzo agent

In [53]:
files_directory = "C:/Users/g.nenna/OneDrive - Reply/Documents/GitHub/hakapizza/Hackapizza Dataset/Misc/"
document_agent_3 = get_txt_files(files_directory)[0]

In [54]:
@tool
def query_document_diverso(query):
    """query the document and return the relevant information"""
    prompt = """ given a query about a document, return the relevant information
    DOCUMENT:
        {document}
    QUERY:
        {query}
    """
    #print(prompt.format(document=document_agent_2, query=query))
    return llm_openai.invoke([HumanMessage(prompt.format(document=document_agent_3, query=query))]).content

document_agent_3_prompt = """You are an expert in extracting information from a document that has the following characteristics: . questo documento è un'affascinante esplorazione di tecniche culinarie avanzate in un contesto fantascientifico! Immagino che le sfide e le opportunità offerte dalla cucina nello spazio possano stimolare moltissime idee creative e innovative. 

Se dovessi riassumere i punti chiave del documento, sembra che copra:

1. **Presentazione dello Chef** - Sirius Cosmo si presenta come una figura di spicco nella cucina galattica e introduce il lettore al tema del manuale: cucinare nello spazio.

2. **Abilità e Licenze** - La sezione descrive le qualifiche necessarie per cucinare nello spazio, incluse abilità psioniche, temporali, gravitazionali e quantistiche.

3. **Ordini Culinari** - Differenti ordini culinari che rappresentano filosofie e approcci diversi nel cucinare, dall'Ordine della Galassia di Andromeda, che aborre il lattosio, ai Naturalisti, che enfatizzano la purezza degli ingredienti.

4. **Tecniche di Preparazione** - Varie tecniche sono esplorate, quali marinatura, affumicatura e fermentazione, con approcci unici come l'uso di antimateria, psionica e tecnologie quantistiche.

5. **Tecniche di Cottura** - Inclusi metodi come bollitura, grigliare, cuocere al forno e vapore, adattati alle condizioni spaziali e con sottosistemi tecnologici avanzati.

6. **Tecniche di Impasto e Surgelamento** - L'ampia gamma di metodi innovativi applicati per impastare e surgelare assicurano la qualità e perfezione degli ingredienti.

7. **Tecniche Avanzate** - Tecniche più esotiche e complesse come la decostruzione interdimensionale e la sferificazione psionica.

Il documento evoca un ambiente di cucina interstellare dove ogni aspetto del processo culinario è amplificato dalla tecnologia e abilità quasi magiche. Probabilmente una delizia per chi ama sia la buona cucina che la fantascienza!
   """

expert_agent_3 = create_react_agent(llm_openai, [query_document_diverso], state_modifier=SystemMessage(document_agent_3_prompt))

In [55]:
expert_agent_3.invoke({'messages': [('user', 'info su licenza elenca alcune tecniche avanzate')]})

{'messages': [HumanMessage(content='info su licenza elenca alcune tecniche avanzate', additional_kwargs={}, response_metadata={}, id='069f74d0-326a-4220-8db2-b4d064de5f71'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_TlHmvzhQVOhpHQFzy2QHNhXu', 'function': {'arguments': '{"query": "licenza per cucinare nello spazio"}', 'name': 'query_document_diverso'}, 'type': 'function'}, {'id': 'call_PqVW31Mdj4nmQ5fYaDJ8mbhS', 'function': {'arguments': '{"query": "tecniche avanzate di cucina spaziale"}', 'name': 'query_document_diverso'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 62, 'prompt_tokens': 540, 'total_tokens': 602, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_4691090a87', 'finish_reason': 't

In [133]:
summaries = {
    "find_reachable_planets" : "This tool is specialized in finding reachable planets and should be used together with other tools to answer questions about distances and planets reachable within certain light years.",
    "expert_agent_1": "Expert agent in galactic dishes and restaurants. Use this tool to filter a database of dishes and restaurants. Restaurants contain information about chefs (technical certifications, etc.), but do not contain information about distances or light years.",
    "expert_agent_2": "This tool is specialized in extracting information from a document that deals with regulated substances and preparation licenses. It does not contain information about distances or light years.",
    "expert_agent_3": "This tool is specialized in extracting information from a document that deals with space cuisine and advanced culinary techniques. It does not contain information about distances or light years.",
}
main_agent_prompt = f"""
        You are an assistant that needs to respond to a user request.
        Plan and write an execution plan to respond to the user's request.
        To do so, you must coordinate the work of the available tools.
        Each tool responds to natural language questions and is specialized
        in a particular topic.
        Below you will find a description of each tool and what information you can obtain from each of them.
        {summaries}
 
        The goal is always and only to return the name of one or more dishes.
       
        To respond to the user's request, you must follow these steps:
        - Based on the descriptions of each tool, break down the question and contact the appropriate tools
        - If a tool does not return results, try contacting another tool
        - Combine all obtained results into a single response
        - If necessary, use the output of one tool to formulate the question for another tool
        - If a tool does not return the expected results, try contacting another tool

        Example reasoning:
        question:
        Which dishes include Spaghi del Sole and are prepared using at least one freezing technique from Sirius Cosmo, 150 light years away from Kylos?
        reasoning:
        - I contact expert agent 1 to get dishes that include Spaghi del Sole
        - I contact expert agent 3 to get freezing techniques from Sirius Cosmo
        - I contact find_reachable_planets to check planets reachable within 150 light years from Kylos
        - I combine the results and return the dishes that satisfy both conditions

        Use a step-by-step approach and only contact the tools when necessary to reach the perfect solution.
"""

In [130]:
print(main_agent_prompt)


        You are an assistant that needs to respond to a user request.
        To do so, you must coordinate the work of the available tools.
        Each tool responds to natural language questions and is specialized
        in a particular topic.
        Below you will find a description of each tool and what information you can obtain from each of them.
        {'find_reachable_planets': 'This tool is specialized in finding reachable planets and should be used together with other tools to answer questions about distances and planets reachable within certain light years.', 'expert_agent_1': 'Expert agent in galactic dishes and restaurants. Use this tool to filter a database of dishes and restaurants. Restaurants contain information about chefs (technical certifications, etc.), but do not contain information about distances or light years.', 'expert_agent_2': 'This tool is specialized in extracting information from a document that deals with regulated substances and preparation licens

main_agent_prompt

In [136]:
## main agent


@tool
def expert_agent_1_tool(query):
    """tool to query the first expert agent. He is an expert in dishes and restaurants. If you want to know what dishes include a specific ingredient or the different techniques used for dishes and many other things you can query this tool"""
    return expert_agent_1.invoke({"messages": [("user", query)]})['messages'][-1].content

@tool
def expert_agent_2_tool(query):
    """tool to query the second expert agent. He is an expert in regulamented substances and preparation licenses. You can query this tool if you want to know more about the substances used in the dishes or the licenses needed to prepare them but not about the dishes themselves"""
    return expert_agent_2.invoke({"messages": [("user", query)]})['messages'][-1].content

@tool
def expert_agent_3_tool(query):
    """tool to query the third expert agent. He is an expert in advanced culinary techniques so you can query this tool if you want to know more about techniques used but not about the dishes themselves"""
    return expert_agent_3.invoke({"messages": [("user", query)]})['messages'][-1].content

graph = {
    "Tatooine": [("Asgard", 695), ("Namecc", 641), ("Arrakis", 109), ("Krypton", 661),
                 ("Pandora", 1130), ("Cybertron", 344), ("Ego", 835), ("Montressosr", 731), ("Klyntar", 530)],
    "Asgard": [("Tatooine", 695), ("Namecc", 550), ("Arrakis", 781), ("Krypton", 188),
               ("Pandora", 473), ("Cybertron", 493), ("Ego", 156), ("Montressosr", 240), ("Klyntar", 479)],
    "Namecc": [("Tatooine", 641), ("Asgard", 550), ("Arrakis", 651), ("Krypton", 367),
               ("Pandora", 987), ("Cybertron", 728), ("Ego", 688), ("Montressosr", 767), ("Klyntar", 845)],
    "Arrakis": [("Tatooine", 109), ("Asgard", 781), ("Namecc", 651), ("Krypton", 727),
                ("Pandora", 1227), ("Cybertron", 454), ("Ego", 926), ("Montressosr", 834), ("Klyntar", 640)],
    "Krypton": [("Tatooine", 661), ("Asgard", 188), ("Namecc", 367), ("Arrakis", 727),
                ("Pandora", 626), ("Cybertron", 557), ("Ego", 321), ("Montressosr", 422), ("Klyntar", 599)],
    "Pandora": [("Tatooine", 1130), ("Asgard", 473), ("Namecc", 987), ("Arrakis", 1227),
                ("Krypton", 626), ("Cybertron", 847), ("Ego", 317), ("Montressosr", 413), ("Klyntar", 731)],
    "Cybertron": [("Tatooine", 344), ("Asgard", 493), ("Namecc", 728), ("Arrakis", 454),
                  ("Krypton", 557), ("Pandora", 847), ("Ego", 594), ("Montressosr", 434), ("Klyntar", 186)],
    "Ego": [("Tatooine", 835), ("Asgard", 156), ("Namecc", 688), ("Arrakis", 926),
            ("Krypton", 321), ("Pandora", 317), ("Cybertron", 594), ("Montressosr", 215), ("Klyntar", 532)],
    "Montressosr": [("Tatooine", 731), ("Asgard", 240), ("Namecc", 767), ("Arrakis", 834),
                    ("Krypton", 422), ("Pandora", 413), ("Cybertron", 434), ("Ego", 215), ("Klyntar", 331)],
    "Klyntar": [("Tatooine", 530), ("Asgard", 479), ("Namecc", 845), ("Arrakis", 640),
                ("Krypton", 599), ("Pandora", 731), ("Cybertron", 186), ("Ego", 532), ("Montressosr", 331)]
}
 
@tool
def find_reachable_planets(start, max_distance):
    """tool to find reachable planets given a starting planet and a max distance"""
    if start not in graph:
        return []
    res = [planet for planet, distance in graph[start] if distance <= max_distance]
    res.insert(0, (start,0))
    return res
 

main_agent = create_react_agent(llm_openai, [expert_agent_1_tool, expert_agent_2_tool, expert_agent_3_tool, find_reachable_planets], state_modifier=SystemMessage(main_agent_prompt),)

TypeError: create_react_agent() got an unexpected keyword argument 'verbose'

In [137]:
main_agent.invoke({'messages': [('user', 'Quali piatti sono stati creati utilizzando almeno una tecnica di surgelamento secondo il di Sirius Cosmo e sono stati serviti in un ristorante situato entro un raggio di 317 anni luce dal pianeta Krypton, Krypton incluso?')]})


SELECT dish_name 
FROM dishes 
WHERE restaurant_name IN (
    SELECT restaurant_name 
    FROM restaurants 
    WHERE location = 'Krypton'
) 
AND techniques LIKE '%Cryo-Tessitura Energetica Polarizzata%' 
OR techniques LIKE '%Congelamento Bio-Luminiscente Sincronico%' 
OR techniques LIKE '%Cristallizzazione Temporale Reversiva%' 
OR techniques LIKE '%Congelazione Iperdimensionalmente Stratificata%' 
OR techniques LIKE '%Surgelamento Antimaterico a Risonanza Inversa%'


SELECT dish_name 
FROM dishes 
WHERE restaurant_name IN (
    SELECT restaurant_name 
    FROM restaurants 
    WHERE location = 'Asgard'
) 
AND techniques LIKE '%Cryo-Tessitura Energetica Polarizzata%' 
OR techniques LIKE '%Congelamento Bio-Luminiscente Sincronico%' 
OR techniques LIKE '%Cristallizzazione Temporale Reversiva%' 
OR techniques LIKE '%Congelazione Iperdimensionalmente Stratificata%' 
OR techniques LIKE '%Surgelamento Antimaterico a Risonanza Inversa%'



{'messages': [HumanMessage(content='Quali piatti sono stati creati utilizzando almeno una tecnica di surgelamento secondo il di Sirius Cosmo e sono stati serviti in un ristorante situato entro un raggio di 317 anni luce dal pianeta Krypton, Krypton incluso?', additional_kwargs={}, response_metadata={}, id='aba68439-420b-4967-8fb8-7a974c9f9d24'),
  AIMessage(content="Per rispondere alla tua domanda, seguirò questi passaggi:\n\n1. **Identificare le tecniche di surgelamento di Sirius Cosmo**: Utilizzerò l'agente esperto 3 per ottenere informazioni sulle tecniche di surgelamento utilizzate da Sirius Cosmo.\n\n2. **Trovare i pianeti raggiungibili entro 317 anni luce da Krypton**: Utilizzerò lo strumento per trovare i pianeti raggiungibili per ottenere un elenco di pianeti entro 317 anni luce da Krypton, incluso Krypton stesso.\n\n3. **Identificare i piatti serviti nei ristoranti situati sui pianeti raggiungibili**: Utilizzerò l'agente esperto 1 per ottenere un elenco di piatti serviti nei r

In [96]:
import pandas as pd
questions = pd.read_csv("Hackapizza Dataset/domande.csv")['domanda'].tolist()
responses = []
for question in questions:
    main_agent = create_react_agent(llm_openai, [expert_agent_1_tool, expert_agent_2_tool, expert_agent_3_tool], state_modifier=SystemMessage(main_agent_prompt))
    response = main_agent.invoke({"messages": [("user", question)]})['messages'][-1].content
    responses.append(response)

responses



SELECT dish_name 
FROM dishes 
WHERE ingredients LIKE '%Chocobo Wings%'


SELECT dish_name 
FROM dishes 
WHERE ingredients LIKE '%Cioccorane%'


SELECT dish_name FROM dishes WHERE ingredients LIKE '%Latte%'


SELECT dish_name 
FROM dishes 
WHERE ingredients LIKE '%Ravioli al Vaporeon%'


SELECT dish_name FROM dishes WHERE ingredients LIKE '%Sashimi di Magikarp%'


SELECT dish_name 
FROM dishes 
WHERE ingredients LIKE '%Frutti del Diavolo%'


SELECT dish_name 
FROM dishes 
WHERE techniques LIKE '%Grigliatura a Energia Stellare DiV%'


SELECT dish_name 
FROM dishes 
WHERE techniques LIKE '%Sferificazione a Gravità Psionica Variabile%'


SELECT dish_name 
FROM dishes 
WHERE techniques LIKE '%Marinatura Temporale Sincronizzata%'


SELECT dish_name 
FROM dishes 
WHERE techniques LIKE '%Congelamento Bio-Luminiscente Sincronico%'


SELECT dish_name FROM dishes WHERE techniques LIKE '%ethereal%'


SELECT dish_name 
FROM dishes 
WHERE ingredients LIKE '%Riso di Cassandra%' 
AND ingredients LIK

['Il piatto che include le "Chocobo Wings" come ingrediente si trova nel ristorante "Galassia di Sapori: Il Viaggio Senza Tempo".',
 'Per un banchetto a tema magico che includa le celebri Cioccorane, ti consiglio di scegliere la "Sinfonia Cosmica: Versione Pizza".',
 'Il piatto della galassia che contiene "Latte+" è la "Pizza Cosmica all\'Essenza di Drago con Nebbia Arcobaleno e Funghi Orbitali".',
 'Il piatto che contiene i "Ravioli al Vaporeon" è la "Sinfonia Cosmica di Sapore".',
 'Non ci sono informazioni specifiche sui piatti che includono i "Sashimi di Magikarp". Potrebbe trattarsi di un piatto immaginario o non dettagliato nei documenti disponibili.',
 'Il piatto "Rinascita Cosmica" è accompagnato dai misteriosi Frutti del Diavolo, che donano poteri speciali a chi li consuma.',
 'I piatti preparati con la tecnica "Grigliatura a Energia Stellare DiV" sono:\n\n1. **Rivisitazione del Kraken sotto Molecole**\n2. **Stella Nova**\n3. **Portale del Cosmo: Sinfonia di Sapori Multidimens

In [97]:
#loads mapping.json
with open('Hackapizza Dataset/Misc/dish_mapping.json', 'r') as file:
    dish_mapping = json.load(file)

dishes = list(dish_mapping.keys())
prompt = f"Data questa lista di piatti, estrai i campi 'nome piatto', anche scritti in inglese e converti nell'intero equivalente seguendo il mapping fornito\n{dish_mapping}. Se per una riga sono presenti più nomi, separali con una virgola. Se non è presente alcun valore inventalo pescando fra quelli presenti nel mapping."
prompt = """
Dato questo elenco di piatti e data una frase, estrai tutti i piatti presenti nella frase che compaiono anche nel mapping fornito. Nel caso compaiano più nomi separali con una virgola. Se non è presente alcun valore inventalo pescando fra quelli presenti nel mapping.
ELENCO:{dishes}
 FRASE: {sentence}
 ESEMPIO DI OUTPUT:
 [PIATTO1, PIATTO2, PIATTO3]
"""
mapping_result = []
i = 0
for sentence in responses:
    print(i)
    i += 1
    res = llm_openai.invoke([HumanMessage(prompt.format(dishes=dishes, sentence=sentence))]).content
    mapping_result.append(res)



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [98]:
final_result = []
for index, row in enumerate(mapping_result):
    final_result.append([])
    elements = row.strip('[]').split(', ')
    for element in elements:
        if element in dish_mapping.keys():
            final_result[index].append(dish_mapping[element])
        else:
            final_result[index].append(42)
final_result

[[78],
 [225],
 [156],
 [215],
 [0],
 [179],
 [189, 267, 171],
 [51, 209, 6, 13, 130, 15],
 [220, 82, 57, 64, 26],
 [0],
 [104],
 [185, 254, 86, 217, 138],
 [127,
  233,
  54,
  16,
  55,
  166,
  162,
  86,
  20,
  247,
  286,
  179,
  156,
  123,
  58,
  88,
  2,
  42,
  125,
  104,
  121,
  235,
  201,
  176,
  197,
  114,
  41,
  275,
  276,
  266,
  1,
  234,
  200,
  216,
  39,
  264,
  163,
  33,
  145,
  252,
  282,
  102,
  149,
  206],
 [221,
  233,
  187,
  166,
  160,
  241,
  64,
  211,
  125,
  168,
  272,
  75,
  208,
  266,
  1,
  226,
  69,
  68,
  101,
  81,
  277,
  267],
 [160, 191, 281, 248, 34, 269],
 [160, 191, 217, 281, 248, 34],
 [0, 1, 3],
 [127, 221, 5, 233, 85, 87, 151, 220, 144, 54],
 [0],
 [183],
 [241, 168],
 [51, 129],
 [127,
  220,
  54,
  89,
  202,
  198,
  99,
  191,
  20,
  48,
  57,
  167,
  150,
  25,
  255,
  104,
  126,
  284,
  251,
  117,
  249,
  100,
  108,
  175,
  271,
  204,
  173,
  13,
  79,
  1,
  226,
  234,
  69,
  214,
  109,
  68,


In [99]:
len(final_result)
df = pd.DataFrame(columns=['row_id', 'result'])
for index, row in enumerate(final_result):
    string = str(row).replace('[', '').replace(']', '')
    print(string)
    df.loc[len(df)] = {'row_id': index+1, 'result': string }
df.to_csv('results/mapping.csv', index=False)

78
225
156
215
0
179
189, 267, 171
51, 209, 6, 13, 130, 15
220, 82, 57, 64, 26
0
104
185, 254, 86, 217, 138
127, 233, 54, 16, 55, 166, 162, 86, 20, 247, 286, 179, 156, 123, 58, 88, 2, 42, 125, 104, 121, 235, 201, 176, 197, 114, 41, 275, 276, 266, 1, 234, 200, 216, 39, 264, 163, 33, 145, 252, 282, 102, 149, 206
221, 233, 187, 166, 160, 241, 64, 211, 125, 168, 272, 75, 208, 266, 1, 226, 69, 68, 101, 81, 277, 267
160, 191, 281, 248, 34, 269
160, 191, 217, 281, 248, 34
0, 1, 3
127, 221, 5, 233, 85, 87, 151, 220, 144, 54
0
183
241, 168
51, 129
127, 220, 54, 89, 202, 198, 99, 191, 20, 48, 57, 167, 150, 25, 255, 104, 126, 284, 251, 117, 249, 100, 108, 175, 271, 204, 173, 13, 79, 1, 226, 234, 69, 214, 109, 68, 118, 39, 264, 280, 44, 103, 213, 164, 183, 0, 282, 149, 112, 222, 17, 113, 177, 269
62, 9, 11, 254, 251
0
0
187, 243, 115, 17, 92
54, 62, 243, 205, 17
54, 225
180, 110, 112, 190, 205
0
206
127, 221, 5, 233, 85, 87, 151, 220, 144, 54
272
198, 217, 241, 247, 67, 255, 2, 246, 244, 104, 223,